In [1]:
import pandas as pd
import numpy as np
import unicodedata
import re

In [2]:
data = pd.read_csv('Datos Madrid/Datos Madrid.csv')

### Eliminación de registros sin información

En caso de presentar valores desconocidos en los campos Comentario, Mas detalles e Info. Será imposible
trabajar con esos registros, ya que no hay forma de obtener información de ellos.
Estos registros suelen corresponder a anuncios retirados que ya no disponían de información
a la hora de relaizar el raspado.

In [3]:
# Comprobamos los datos que cumplen esta condición

columns_to_study = ['Comentario', 'Mas detalles', 'Info']

incomplete_condition = (data[columns_to_study] == 'Desconocido').all(axis = 1)
data_incomplete = data[incomplete_condition]

In [4]:
print(data_incomplete.sample(8),'\n\n') # Comprobamos que están incompletos los campos

print('Cantidad de datos inservibles: ',len(data_incomplete)) # Visualizamos el total de datos incompletos

             ID       Precio Planta o Tipo N Habitaciones           m2  \
3667   97032821  Desconocido   Desconocido    Desconocido  Desconocido   
14972  97227176  Desconocido   Desconocido    Desconocido  Desconocido   
14195  97215533  Desconocido   Desconocido    Desconocido  Desconocido   
4242   97206424  Desconocido   Desconocido    Desconocido  Desconocido   
14729  95888397  Desconocido   Desconocido    Desconocido  Desconocido   
8797   97310512  Desconocido   Desconocido    Desconocido  Desconocido   
12175  96313577  Desconocido   Desconocido    Desconocido  Desconocido   
12241  97350196  Desconocido   Desconocido    Desconocido  Desconocido   

        Comentario                 Zona           Subzona         Info  \
3667   Desconocido            Chamartin           El Viso  Desconocido   
14972  Desconocido           Arganzuela           Acacias  Desconocido   
14195  Desconocido          Carabanchel          Abrantes  Desconocido   
4242   Desconocido            Chamart

In [5]:
# Guardamos los valores completos en data2

data = data[~(incomplete_condition)]

In [6]:
len(data)

19351

In [7]:
data = data.reset_index(drop=True)

### Tratamiento de campo Planta o Tipo

In [8]:
# Comprobamos cuántos valores desconocidos hay en esta columna

sum(data['Planta o Tipo'] == 'Desconocido')

2555

In [9]:
def standard_words(sentence):
    return str(unicodedata.normalize('NFKD', sentence.lower()).encode('ASCII', 'ignore'))

def search_words(sentence, words_searched : list):
    return any([word in sentence for word in words_searched])

def search_words_in_df(df : pd.DataFrame, columns_to_study : list, words_searched : list):
    return df[columns_to_study].applymap(lambda x: search_words(x,words_searched)).any(axis = 1)

def is_any_value_filled(df : pd.DataFrame, list_filled : list, column_compare, value_null):
    return sum(list_filled & (df[column_compare] == value_null))

In [10]:
def apply_conditions(data, condition_list, name_fill, column_compare = 'Planta o Tipo', value_null = 'Desconocido'):
    condition_under = np.array(condition_list).reshape((len(condition_list),-1)).any(axis=0)
    values_filled = is_any_value_filled(data, condition_under, column_compare, value_null)
    print(f'Cantidad de valores rellenados con valor {name_fill}: {values_filled}')
    condition_under = condition_under & (data[column_compare]==value_null)
    data.loc[condition_under, column_compare] = name_fill
    return data

def search_number(sentence):
    match = re.search('planta[^0-9]{0,3}(\d{1,})', sentence)
    if match is None:
        return 'Desconocido'
    else:
        return match[1]

In [11]:
data[['Comentario_lower', 'Mas detalles_lower', 'Info_lower']] = data[['Comentario', 'Mas detalles', 'Info']].applymap(lambda x: standard_words(x))

In [12]:
# Correccion de errores en la toma de numeros de planta en el scrapeo
condition_no_under = data['Planta o Tipo'] != 'Bajo'
data.loc[condition_no_under, 'Planta o Tipo'] = data[condition_no_under]['Info_lower'].apply(lambda x: search_number(x))
print(np.unique(data['Planta o Tipo'], return_counts=True))

(array(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19',
       '2', '20', '22', '3', '4', '5', '6', '60', '7', '8', '9', 'Bajo',
       'Desconocido'], dtype=object), array([3821,   76,   53,   28,   43,   20,   13,   10,    1,    7,    3,
       3203,    9,    2, 2483, 2008, 1132,  722,    1,  392,  216,  130,
       2423, 2555]))


In [13]:
condition_list = []
words_searched = ['chalet', 'finca', 'casa rural']

condition_list.append(search_words_in_df(data, ['Comentario_lower', 'Mas detalles_lower', 'Info_lower'], words_searched))

data = apply_conditions(data, condition_list, 'Chalet')

Cantidad de valores rellenados con valor Chalet: 1415


In [14]:
# Comprobamos cuántos valores desconocidos nos quedan ahora
np.unique(data['Planta o Tipo'], return_counts=True)

(array(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19',
        '2', '20', '22', '3', '4', '5', '6', '60', '7', '8', '9', 'Bajo',
        'Chalet', 'Desconocido'], dtype=object),
 array([3821,   76,   53,   28,   43,   20,   13,   10,    1,    7,    3,
        3203,    9,    2, 2483, 2008, 1132,  722,    1,  392,  216,  130,
        2423, 1415, 1140]))

In [15]:
# Otro tipo de inmueble habitual y característico serían las entreplantas, observando  el campo Info
# y Mas detalles, se aprecia que, para este tipo de propiedades, se suele detallar en estos campos
# Siguiendo un proceso similar a los anteriores:
condition_list = []
words_searched = ['este local comercial', 'de uso comercial', 'es local comercial', 'no acondicionado', 
                  'para acondicionar', 'a acondicionar', 'de oficinas', 'uso industrial', 'local industrial']

condition_list.append(search_words_in_df(data, ['Comentario_lower'], words_searched))


words_searched = ['Local comercial']
condition_list.append(search_words_in_df(data, ['Comentario'], words_searched))

data = apply_conditions(data, condition_list, 'Uso comercial')

Cantidad de valores rellenados con valor Uso comercial: 10


In [16]:
condition_list = []
words_searched = ['luminosa buhardilla', 'preciosa buhardilla', 'bonita buhardilla', 'luminoso atico', 
                  'bonito atico', 'precioso atico', 'luminosa azotea', 'preciosa azotea', 'excepcional atico', 
                  'excepcional atico', 'bonita azotea', 'azotea en venta', 'azotea a la venta', 'se vende azotea', 
                  'atico en venta', 'atico a la venta', 'se vende atico', 'buhardilla en venta', 'buhardilla a la venta', 
                  'se vende buhardilla', 'venta de atico', 'venta de azotea', 'piso azotea', 'piso atico', 
                  'venta de buhardilla']

condition_list.append(search_words_in_df(data, ['Comentario_lower'], words_searched))

words_searched = ['Atico', 'Azotea', 'Buhardilla', 'Ultima planta']
condition_list.append(search_words_in_df(data, ['Comentario'], words_searched))

words_searched = ['azotea', 'atico']
condition_list.append(search_words_in_df(data, ['Info_lower'], words_searched))

data = apply_conditions(data, condition_list, 'Atico')

Cantidad de valores rellenados con valor Atico: 7


In [17]:
# Otro tipo de inmueble habitual y característico serían las entreplantas, observando  el campo Info
# y Mas detalles, se aprecia que, para este tipo de propiedades, se suele detallar en estos campos
# Siguiendo un proceso similar a los anteriores:
condition_list = []
words_searched = ['en entreplanta', 'en la entreplanta', 'esta entreplanta', 'piso en entreplanta', 'inmueble en entreplanta',
                 'se trata de una entreplanta', 'en una entreplanta']

condition_list.append(search_words_in_df(data, ['Comentario_lower'], words_searched))


words_searched = ['entreplanta']
condition_list.append(search_words_in_df(data, ['Info_lower', 'Mas detalles_lower'], words_searched))

data = apply_conditions(data, condition_list, 'Entreplanta')

Cantidad de valores rellenados con valor Entreplanta: 204


In [18]:
condition_list = []
words_searched = ['vende sotano', 'vende semisotano', 'es un sotano', 'es un bajo', 'es el bajo', 'se trata de un bajo', 
                'es un semisotano', 'vende un semisotano', 'vende un sotano', 'es una planta baja', 'es la planta baja', 
                'es planta baja', 'en planta baja', 'es sotano', 'es semisotano', 'es el sotano', 'es el semisotano',
                'en el sotano', 'en el semisotano', 'en la planta baja', 'semisotano a la venta', 'semisotano en venta', 
                 'sotano a la venta', 'sotano en venta', 'semisotano exterior', 'sotano exterior', 'planta baja exterior', 
                'bonito sotano', 'estupendo sotano', 'precioso sotano', 'magnifico sotano', 'en un semisotano', 
                'en planta sotano', 'semi sotano', 'en un sotano', 'en una planta baja', 'en planta semisotano', 
                'en la planta sotano', 'en la planta semisotano', 'piso en planta baja', 'vivienda en planta baja', 
                'inmueble en sotano', 'piso en sotano', 'vivenda en sotano', 'inmueble en semisotano', 'piso en semisotano', 
                'vivenda en semisotano', 'inmueble en semi sotano', 'piso en semi sotano', 'vivenda en semi sotano', 
                'inmueble en semi-sotano', 'piso en semi-sotano', 'vivenda en semi-sotano']

condition_list.append(search_words_in_df(data, ['Comentario_lower'], words_searched))


words_searched = ['Bajo', 'Planta baja', 'Sotano', 'Semisotano', 'Semi-sotano', 'esta en la planta baja', 
                  'Ubicada en la planta baja']
condition_list.append(search_words_in_df(data, ['Comentario'], words_searched))

words_searched = ['semisotano', 'semi sotano', 'semi-sotano']
condition_list.append(search_words_in_df(data, ['Info_lower', 'Mas detalles_lower'], words_searched))

data = apply_conditions(data, condition_list, 'Bajo') 

Cantidad de valores rellenados con valor Bajo: 213


In [19]:
condition_list = []
words_searched = ['apartamento', 'estudio', 'piso a la venta', 'vende piso', 
                'estupendo piso', 'precioso piso', 'espacioso piso', 'esplendido piso', 'luminoso piso', 
                'fantastico piso', 'excelente piso', 'magnifico piso', 'fabuloso piso', 'comodo piso', 
                'amplio piso', 'piso estupendo', 'piso precioso', 'piso espacioso', 'piso esplendido', 
                'piso luminoso ', 'piso fantastico ', 'piso perfecto ', 'piso excelente', 'piso magnifico', 
                'fabuloso piso', 'piso comodo', 'piso amplio', 'piso maravilloso ', 'piso exterior', 'piso interior', 
                'piso reformado', 'piso con reforma', 'piso a reformar', 'loft', 'duplex', 'triplex', 'piso de', 
                'piso ubicado', 'piso situado', 'lujoso piso', 'piso totalmente', 'piso en ', 'piso piloto', 'piso muy', 
                'piso con', 'piso situado', 'piso reformado', 'piso sin reformar', 'piso a reformar', 'piso para reformar', 
                'piso a falta de reformar', 'piso bastante']
condition_list.append(search_words_in_df(data, ['Comentario_lower'], words_searched))

data = apply_conditions(data, condition_list, 'Piso') 


Cantidad de valores rellenados con valor Piso: 505


In [20]:
condition_list = []
words_searched = ['viviendas', 'complejo residencial', 'inmuebles', 'pisos', 'apartamentos']
condition_list.append(search_words_in_df(data, ['Comentario_lower'], words_searched))

words_searched = ['estudio']
condition_list.append(search_words_in_df(data, ['Info_lower', 'Mas detalles_lower'], words_searched))
data = apply_conditions(data, condition_list, 'Residencial') 

Cantidad de valores rellenados con valor Residencial: 86


In [21]:
condition_list = []
words_searched = ['primera planta', 'segunda planta', 'tercera planta', 'cuarta planta', 
                 'quinta planta', 'sexta planta', 'septima planta', 'octava planta', 
                 'novena planta', 'decima planta', 'planta primera', 'planta segunda', 
                 'planta tercera', 'planta cuarta', 'planta quinta', 'planta sexta', 
                 'planta septima', 'planta octava', 'primer piso', 'segundo piso', 'tercer piso', 
                 'cuarto piso', 'quinto piso', 'sexto piso', 'septimo piso', 'octavo piso', 
                 'noveno piso', 'decimo piso', 'con ascensor', 'de ascensor']

condition_list.append(search_words_in_df(data, ['Comentario_lower'], words_searched))

words_searched = ['ascensor']

condition_list.append(search_words_in_df(data, ['Mas detalles_lower'], words_searched))

data = apply_conditions(data, condition_list, 'Piso') 

Cantidad de valores rellenados con valor Piso: 107


In [22]:
# Comprobamos que ya no quedan valores desconocidos

data.loc[data['Planta o Tipo'] == 'Desconocido', 'Planta o Tipo'] = 'Piso'

## Tratamiento de plantas con nuevo campo piso

Creamos un nuevo campo Planta para detallar la altura de la planta del piso en oferta, dejaremos el campo Planta o Tipo como Tipo

In [23]:
# Creamos un nuevo dataset llamado df y renombramos la columna Planta o Tipo por solamente Tipo

data.rename(columns={'Planta o Tipo': 'Tipo'}, inplace=True)

In [24]:
# Creamos la nueva columna Piso con valores cero por defecto

data['Planta'] = data['Tipo'].apply(lambda x: x if re.search('[0-9]', x) is not None else "0")
data['Tipo'] = data['Tipo'].apply(lambda x: "Piso" if re.search('[0-9]', x) is not None else x)

In [25]:
# Buscamos palabras y expresiones habituales para los cada piso


#### PLANTA 1 ####
# Búsqueda en campo comentario en minúsculas ignorando mayúsculas
condition_list = []
words_searched = ['primera planta', '1a planta', 'planta 1 ', 'piso 1 ', '1er piso', 'primer piso', 
                  'planta primera', 'piso primero', 'un primer', 'un primero', 
                  'el primer', 'el primer', 'una primera', 'la primera', '1a', '1o', 'un primero']
condition_list.append(search_words_in_df(data, ['Comentario_lower'], words_searched))
data = apply_conditions(data, condition_list, '1', 'Planta', '0')


### PLANTA 2 ####
# Búsqueda en campo comentario en minúsculas ignorando mayúsculas
condition_list = []
words_searched = ['segunda planta' , '2a planta', '2o piso', 'planta 2 ', 'piso 2 ' , 
                  'segundo piso', 'planta segunda', 'piso segundo', 'un segundo', 
                  'el segundo', 'una segunda', 'la segunda', '2o', '2a']
condition_list.append(search_words_in_df(data, ['Comentario_lower'], words_searched))
data = apply_conditions(data, condition_list, '2', 'Planta', '0')


### PLANTA 3 ####
# Búsqueda en campo comentario en minúsculas ignorando mayúsculas
condition_list = []
words_searched = ['tercera planta', '3a planta', '3o piso', 'planta 3 ', 'piso 3 ', 'tercer piso', 'planta tercera', 'piso tercero', 'un tercer', 'el tercer', 'el tercero', 'una tercera', 'la tercera', '3o', '3a']
condition_list.append(search_words_in_df(data, ['Comentario_lower'], words_searched))
data = apply_conditions(data, condition_list, '3', 'Planta', '0')

# Búsqueda en campo comentario normal, con signos literales del anuncio
condition_list = []
words_searched = ['3ª', '3º']
condition_list.append(search_words_in_df(data, ['Comentario'], words_searched))
data = apply_conditions(data, condition_list, '3', 'Planta', '0')

### PLANTA 4 ####
# Búsqueda en campo comentario en minúsculas ignorando mayúsculas
condition_list = []
words_searched = ['cuarta planta' , '4a planta', '4o piso', 'planta 4 ', 'piso 4 ' , 'cuarto piso', 'planta cuarta', 'piso cuarto', 'un cuarto', 'el cuarto', 'una cuarta', 'la cuarta', '4o', '4a']
condition_list.append(search_words_in_df(data, ['Comentario_lower'], words_searched))
data = apply_conditions(data, condition_list, '4', 'Planta', '0')


### PLANTA 5 ####
# Búsqueda en campo comentario en minúsculas ignorando mayúsculas
condition_list = []
words_searched = ['quinta planta' , '5a planta', '5o piso', 'planta 5 ', 'piso 5 ' , 'quinto piso', 'planta quinta', 'piso quinto', 'un quinto', 'el quinto', 'una quinta', 'la quinta', '5o', '5a']
condition_list.append(search_words_in_df(data, ['Comentario_lower'], words_searched))
data = apply_conditions(data, condition_list, '5', 'Planta', '0')


### PLANTA 6 ####
# Búsqueda en campo comentario en minúsculas ignorando mayúsculas
condition_list = []
words_searched = ['sexta planta' , '6a planta', '6o piso' , 'sexto piso', 'planta sexta', 'piso sexto', 'un sexto', 'el sexto', 'una sexta', 'la sexta', '6o', '6a']
condition_list.append(search_words_in_df(data, ['Comentario_lower'], words_searched))
data = apply_conditions(data, condition_list, '6', 'Planta', '0')


### PLANTA 7 ####
# Búsqueda en campo comentario en minúsculas ignorando mayúsculas
condition_list = []
words_searched = ['septima planta' , '7a planta', '7o piso', 'planta 7 ', 'piso 7 ', 'septimo piso', 'planta septima', 'piso septimo', 'un septimo', 'el septimo', 'una septima', 'la septima', '7o', '7a']
condition_list.append(search_words_in_df(data, ['Comentario_lower'], words_searched))
data = apply_conditions(data, condition_list, '7', 'Planta', '0')


### PLANTA 8 ####
# Búsqueda en campo comentario en minúsculas ignorando mayúsculas
condition_list = []
words_searched = ['octava planta' , '8a planta', '8o piso', 'planta 8 ', 'piso 8 ' , 'octavo piso', 'planta octava', 'piso octavo', 'un octavo', 'el octavo', 'una octava', 'la octava', '8o', '8a' ]
condition_list.append(search_words_in_df(data, ['Comentario_lower'], words_searched))
data = apply_conditions(data, condition_list, '8', 'Planta', '0')


### PLANTA 9 ####
# Búsqueda en campo comentario en minúsculas ignorando mayúsculas
condition_list = []
words_searched = ['novena planta' , '9a planta', '9o piso', 'planta 9 ', 'piso 9 ', 'noveno piso', 'planta novena', 'piso noveno', 'un noveno', 'el noveno', 'una novena', 'la novena', '9o', '9a']
condition_list.append(search_words_in_df(data, ['Comentario_lower'], words_searched))
data = apply_conditions(data, condition_list, '9', 'Planta', '0')


### PLANTA 10 ####
# Búsqueda en campo comentario en minúsculas ignorando mayúsculas
condition_list = []
words_searched = ['decima planta' , '10a planta', '10 piso', 'planta 10 ', 'piso 10 ' , 'decimo piso', 'planta decima', 'piso decimo', 'un decimo', 'el decimo', 'una decima', 'la decima', '10o', '10a']
condition_list.append(search_words_in_df(data, ['Comentario_lower'], words_searched))
data = apply_conditions(data, condition_list, '10', 'Planta', '0')


Cantidad de valores rellenados con valor 1: 854
Cantidad de valores rellenados con valor 2: 234
Cantidad de valores rellenados con valor 3: 80
Cantidad de valores rellenados con valor 3: 0
Cantidad de valores rellenados con valor 4: 172
Cantidad de valores rellenados con valor 5: 30
Cantidad de valores rellenados con valor 6: 6
Cantidad de valores rellenados con valor 7: 6
Cantidad de valores rellenados con valor 8: 3
Cantidad de valores rellenados con valor 9: 0
Cantidad de valores rellenados con valor 10: 3


In [26]:
data[(data['Planta'] == '0') & (data['Tipo'] == 'Piso')]

,ID,Precio,Tipo,N Habitaciones,m2,Comentario,Zona,Subzona,Info,Mas detalles,Comentario_lower,Mas detalles_lower,Info_lower,Planta
84,85449897,579.000,Piso,5,131,[Vivienda2 os brinda la oportunidad de inverti...,Centro,Malasaña-Universidad,131 m² 5 hab. exterior con ascensor,"[131 m² construidos, 117 m² útiles, 5 habitaci...",b'[vivienda2 os brinda la oportunidad de inver...,"b'[131 m2 construidos, 117 m2 utiles, 5 habita...",b' 131 m2 5 hab. exterior con ascensor ',0
111,97322351,109.900,Piso,Desconocido,27,[Redpiso pone a la venta Estudio ACONDICIONADO...,Centro,Malasaña-Universidad,27 m² exterior sin ascensor,"[27 m² construidos, Sin habitación, 1 baño, Se...",b'[redpiso pone a la venta estudio acondiciona...,"b'[27 m2 construidos, sin habitacion, 1 bano, ...",b' 27 m2 exterior sin ascensor ',0
153,97286762,156.000,Piso,1,29,"[Redpiso San Bernardo, vende acogedor apartame...",Centro,Malasaña-Universidad,29 m² 1 hab. sin ascensor,"[29 m² construidos, 1 habitación, 1 baño, Segu...","b'[redpiso san bernardo, vende acogedor aparta...","b'[29 m2 construidos, 1 habitacion, 1 bano, se...",b' 29 m2 1 hab. sin ascensor ',0
245,95895760,685.000,Piso,3,103,[Piso reformado a estrenar Engel&amp;Voelkers ...,Centro,Malasaña-Universidad,103 m² 3 hab. exterior sin ascensor,"[103 m² construidos, 93 m² útiles, 3 habitacio...",b'[piso reformado a estrenar engel&amp;voelker...,"b'[103 m2 construidos, 93 m2 utiles, 3 habitac...",b' 103 m2 3 hab. exterior sin ascensor ',0
253,95806565,490.000,Piso,2,74,[Vivienda Reformada en Malasaña Engel&amp;Voel...,Centro,Malasaña-Universidad,74 m² 2 hab. exterior con ascensor,"[74 m² construidos, 57 m² útiles, 2 habitacion...",b'[vivienda reformada en malasana engel&amp;vo...,"b'[74 m2 construidos, 57 m2 utiles, 2 habitaci...",b' 74 m2 2 hab. exterior con ascensor ',0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19284,96323505,466.500,Piso,2,87,"[Magnífico piso de obra nueva en Madrid, perfe...",Hortaleza,Virgen del Cortijo - Manoteras,Obra nueva 87 m² 2 hab.,"[87 m² construidos, 76 m² útiles, 2 habitacion...","b'[magnifico piso de obra nueva en madrid, per...","b'[87 m2 construidos, 76 m2 utiles, 2 habitaci...",b' obra nueva 87 m2 2 hab. ',0
19285,93589718,433.000,Piso,2,87,"[Fantástico piso de obra nueva en Madrid, idea...",Hortaleza,Virgen del Cortijo - Manoteras,Obra nueva 87 m² 2 hab.,"[87 m² construidos, 76 m² útiles, 2 habitacion...","b'[fantastico piso de obra nueva en madrid, id...","b'[87 m2 construidos, 76 m2 utiles, 2 habitaci...",b' obra nueva 87 m2 2 hab. ',0
19286,94211075,497.000,Piso,2,100,"[Fabuloso piso de obra nueva en Madrid, perfec...",Hortaleza,Virgen del Cortijo - Manoteras,Obra nueva 100 m² 2 hab.,"[100 m² construidos, 85 m² útiles, 2 habitacio...","b'[fabuloso piso de obra nueva en madrid, perf...","b'[100 m2 construidos, 85 m2 utiles, 2 habitac...",b' obra nueva 100 m2 2 hab. ',0
19295,95613429,450.000,Piso,2,180,[Colibree presenta asombroso loft reformado in...,Hortaleza,Virgen del Cortijo - Manoteras,180 m² 2 hab. exterior sin ascensor G...,"[180 m² construidos, 2 habitaciones, 2 baños, ...",b'[colibree presenta asombroso loft reformado ...,"b'[180 m2 construidos, 2 habitaciones, 2 banos...",b' 180 m2 2 hab. exterior sin ascensor ...,0


## Creación de campo Cantidad de plantas

Alguno de los pisos que se ofertan poseen en el campo Comentario indicaciones de que se trata de un duplex o triplex en algún caso, guardaremos este dato y lo añadiremos en un campo Cantidad de plantas. Este campo será 1 para todos los pisos excepto aquellos que posean más.

In [27]:
# Buscamos pisos de nuevo, la mayoría de restantes serán de este tipo
# si incluyen el número de planta en el comentario, debería de tratarse de un piso
data['Cantidad de plantas'] = 1

condition_list = []
words_searched = ['duplex', 'vivienda de dos plantas', 'se compone de dos plantas', 'vivienda de dos pisos', 
                  'se compone de dos pisos', 'chale de dos pisos', 'finca de dos pisos', 'chale de dos plantas', 
                  'finca de dos plantas']
condition_list.append(search_words_in_df(data, ['Comentario_lower'], words_searched))
data = apply_conditions(data, condition_list, 2, 'Cantidad de plantas', 1)


condition_list = []
words_searched = ['triplex', 'vivienda de tres plantas', 'se compone de tres plantas', 'vivienda de tres pisos', 
                  'se compone de tres pisos', 'chale de tres pisos', 'finca de tres pisos', 'chale de tres plantas', 
                  'finca de tres plantas']
condition_list.append(search_words_in_df(data, ['Comentario_lower'], words_searched))
data = apply_conditions(data, condition_list, 3, 'Cantidad de plantas', 1)

Cantidad de valores rellenados con valor 2: 978
Cantidad de valores rellenados con valor 3: 49


## Piscina
Comprobación de en donde podemos encontrar el elemento piscina

In [28]:
## Comprobación campo Info
print(sum(data["Info_lower"].apply(lambda x : 1 if 'piscina' in x else 0)))
## Comprobación campo Mas detalles
print(sum(data["Mas detalles_lower"].apply(lambda x : 1 if 'piscina' in x else 0)))
## Comprobación campo Comentario, sin tenr en cuenta los encontrados en Mas detalles
print(sum(data.apply(lambda x : 1 if 'piscina' in x["Comentario_lower"] and 'piscina' not in x["Mas detalles_lower"] else 0, axis = 1)))

0
3100
566


### Mas detalle piscina
Estudio de las apariciones de piscina en el campo Mas detalles extrayendo las dos palabras anteriores para dar contexto

In [29]:
## función que aporta una serie de palabras que rodean la buscada para poder sacar un contexto de su uso.
## Lo que intenta es siempre dar las n(words_for_context) palabbras previas más la buscada, pero en caso de que no pueda
## debido a que la ubicación de la palabra en la frase es menor que las palabras previas que se piden, aportará todas las 
## previas que pueda y las restantes palabras posteriores Ejemplo.- Para la frase "Tiene piscina y garaje",
## si queremos aportar las dos palabras previas a pisina no podemos ya que solo posee 1, por tanto, en este caso nos devolverá:
## "Tiene piscina y"
def search_context(value, sentence, words_for_context = 2):
    any_word = ['[^ ]*']
    for i in range(words_for_context + 1):
        words_before = any_word*(words_for_context - i)
        words_after = any_word*(i)
        regex = words_before + [value] + words_after
        regex = " ".join(regex)
        regex = re.search(regex, sentence)
        if regex is not None:
            return regex[0]
    return "FAIL"

def is_in_good_context(sentence, context_list):
    for context_word in context_list:
        if re.search(context_word, sentence) is not None: 
            return 1
    return 0

In [30]:
data["Mas detalles_lower"].apply(lambda x : search_context("piscina", x) if 'piscina' in x else "0").unique()

array(['0', 'aire acondicionado, piscina', 'con ascensor, piscina',
       'calefaccion individual, piscina', 'sin ascensor, piscina',
       'gas natural, piscina', 'nueva, trastero, piscina',
       'orientacion oeste, piscina', 'en 1962, piscina',
       'movilidad reducida, piscina', 'de calefaccion, piscina',
       'norte, sur, piscina', 'en 1965, piscina',
       'mano/para reformar, piscina', 'central: gasoil, piscina',
       'este, oeste, piscina', 'obra nueva, piscina', 'en 1941, piscina',
       'armarios empotrados, piscina', 'en 2003, piscina',
       'orientacion sur, piscina', 'gas propano/butano, piscina',
       'mano/buen estado, piscina', 'en 2018, piscina',
       'en 2016, piscina', 'de frio/calor, piscina',
       'calefaccion central, piscina', 'en 1966, piscina',
       'en 1992, piscina', 'individual: electrica, piscina',
       'en 2005, piscina', 'estado, trastero, piscina',
       'empotrados, trastero, piscina', 'individual: gasoil, piscina',
       'en 20

La aportación del unique nos da una serie de resultados que permiten catalogar manualmente como positivos todas las entradas que poseen la palabra piscina. 

In [31]:
data["Piscina"] = data["Mas detalles_lower"].apply(lambda x : 1 if 'piscina' in x else 0)

### Comentario piscina
Estudio de los casos en los que aparece piscina en el comentario, pero que todavía no se tienen en cuenta

In [32]:
piscina_comment = data.apply(lambda x : search_context("piscina", x["Comentario_lower"]) if 'piscina' not in x["Mas detalles_lower"] and 'piscina' in x["Comentario_lower"] else "0", axis = 1)
num_ocurrence = np.unique(piscina_comment, return_counts=True)
for item, sentence in enumerate(num_ocurrence[0]):
    print(f'{sentence} appears {num_ocurrence[1][item]}')

  piscina appears 6
 la piscina appears 1
( gofit, piscina appears 1
(25.000)  piscina appears 1
(arboles, jardines, piscina appears 1
(opcional) y piscina appears 5
), colegios, piscina appears 1
0 appears 18785
1.091m2 con piscina appears 1
1.435m2 con piscina appears 1
10, gimnasio, piscina appears 1
2 grandes piscina appears 1
24 horas, piscina appears 2
4 ascensores, piscina appears 1
500 m2, piscina appears 1
50m con piscina appears 1
6 vehiculos. piscina appears 1
80m2 y piscina appears 1
FAIL appears 2
a la piscina appears 11
a lado. piscina appears 1
acceso a piscina appears 1
ajardinada y piscina appears 4
ajardinadas con piscina appears 1
ajardinadas y piscina appears 2
almacenamiento, barbacoa, piscina appears 1
anejo y piscina appears 1
ascensor interior, piscina appears 1
ascensor y piscina appears 14
ascensores,  piscina appears 1
atletismo, polideportivo, piscina appears 1
baloncesto, dos piscina appears 1
cagigal (con piscina appears 1
central con piscina appears 8
cer

Además de aumentar el número de posibles resultados, se intuye que algunos hacen referencia a piscinas cercanas y no de la propia vivienda, así que no se puede dar por hecho que la existencia de la palabra piscina implique que la posea.
Estudiamos los casos más sospechosos a mano.

Empezamos añadiendo los match que posean una palabra clave, que implique que se está hablando de la vivienda en ese contexto:

In [33]:
good_context_list = ["urbanizacion", "garaje", "terraza", "trastero" , "zonas comunes", "ascensor", "sotano", "tendedero"]
data["Piscina"] = data.apply(lambda x : is_in_good_context(search_context('piscina', x["Comentario_lower"], 6), good_context_list) 
                             if x["Piscina"] == 0 and 'piscina' in x["Comentario_lower"] else x["Piscina"], axis = 1)


In [34]:
sum(data[data['Piscina']==1]['Tipo']=='Chalet')

638

Dado el resultado, se da por buena la aproximación de los datos extraidos, aún sabiendo que podría mejorarse. Se considera que el esfuerzo para realizar un filtro mejor no compensa con la poca aportación que daría.

## Ascensor
Comprobación de en donde podemos encontrar el elemento ascensor

In [35]:
element_to_search = 'ascensor'
## Comprobación campo Info
print(sum(data["Info_lower"].apply(lambda x : 1 if element_to_search in x else 0)))
## Comprobación campo Mas detalles, sin tener en cuenta los encontrados en Info
print(sum(data.apply(lambda x : 1 if element_to_search in x["Mas detalles_lower"] and element_to_search not in x["Info_lower"] else 0, axis = 1)))
## Comprobación campo Comentario, sin tenr en cuenta los encontrados en Mas detalles
print(sum(data.apply(lambda x : 1 if element_to_search in x["Comentario_lower"] and element_to_search not in x["Mas detalles_lower"] else 0, axis = 1)))

17908
0
274


### Info ascensor
Estudio de las apariciones de ascensor en el campo Info extrayendo la palabra anterior para dar contexto

In [36]:
data["Info_lower"].apply(lambda x : search_context(element_to_search, x, words_for_context = 1) if element_to_search in x else 0).unique()

array(['con ascensor', 'sin ascensor', 0], dtype=object)

La aportación del unique nos da una serie de resultados que permiten catalogar manualmente como positivos todos las entradas que poseen la palabra con ascensor.

In [37]:
data["Ascensor"] = data["Info_lower"].apply(lambda x : 1 if 'con ascensor' in x else 0)

### Comentario ascensor

Estudio de los casos en los que aparece ascensor en el comentario, pero que todavía no se tienen en cuenta


In [38]:
ascensor_comment = data.apply(lambda x : search_context(element_to_search, x["Comentario_lower"], 1) if element_to_search not in x["Info_lower"] and element_to_search in x["Comentario_lower"] else "0", axis = 1)
num_ocurrence = np.unique(ascensor_comment, return_counts=True)
for item, sentence in enumerate(num_ocurrence[0]):
    print(f'{sentence} appears {num_ocurrence[1][item]}')

 ascensor appears 3
0 appears 19077
automatico, ascensor appears 5
blindados, ascensor appears 1
calidades. ascensor appears 2
coches, ascensor appears 1
como ascensor appears 2
comodidades: ascensor appears 1
comodo ascensor appears 2
con ascensor appears 41
conductos, ascensor appears 1
cristal, ascensor appears 1
de ascensor appears 39
del ascensor appears 2
dia, ascensor appears 1
domotizada. ascensor appears 1
dos ascensor appears 1
el ascensor appears 8
electricos ascensor appears 1
en ascensor appears 1
entrada, ascensor appears 1
estupendo ascensor appears 2
exclusividad. ascensor appears 2
frio/calor. ascensor appears 1
fuerte, ascensor appears 1
gabanero, ascensor appears 1
guardillas ascensor appears 1
hacer ascensor appears 1
hay ascensor appears 1
incluye ascensor appears 2
incorporado. ascensor appears 1
instalado ascensor appears 1
instalar ascensor appears 2
jardin, ascensor appears 1
liso, ascensor appears 1
madera. ascensor appears 1
mediante ascensor appears 1
mejora

Nuevamente en los comentarios, no se ve tan fácil si existe o no ascensor, pero el número de ocurrencias es mucho menor, siendo casi posible realizar la separación manualmente.

In [39]:
good_context_list = ["con", "tiene"]
data["Ascensor"] = data.apply(lambda x : is_in_good_context(search_context(element_to_search, x["Comentario_lower"], 1), good_context_list) 
                             if x["Ascensor"] == 0 and element_to_search in x["Comentario_lower"] else x["Ascensor"], axis = 1)

## Garaje
Comprobación de en donde podemos encontrar el elemento garaje

In [40]:
element_to_search = 'garaje'
## Comprobación campo Info
print(sum(data["Info_lower"].apply(lambda x : 1 if element_to_search in x else 0)))
## Comprobación campo Mas detalles, sin tenr en cuenta los encontrados en Info
print(sum(data.apply(lambda x : 1 if element_to_search in x["Mas detalles_lower"] and element_to_search not in x["Info_lower"] else 0, axis = 1)))
## Comprobación campo Comentario, sin tenr en cuenta los encontrados en Mas detalles
print(sum(data.apply(lambda x : 1 if element_to_search in x["Comentario_lower"] and element_to_search not in x["Mas detalles_lower"] else 0, axis = 1)))

5549
0
1062


### Info garaje
Estudio de las apariciones de garaje en el campo Info extrayendo la palabra anterior para dar contexto

In [41]:
data["Info_lower"].apply(lambda x : search_context(element_to_search, x, words_for_context = 3) if element_to_search in x else 0).unique()

array(['ascensor   garaje', 0, 'hab.   garaje', '2a   garaje',
       'exterior   garaje', '1a   garaje', '3a   garaje', 'm2   garaje'],
      dtype=object)

La aportación del unique nos da una serie de resultados que permiten catalogar manualmente como positivos todos las entradas que poseen la palabra con garaje.

### Comentario garaje

Estudio de los casos en los que aparece garaje en el comentario, pero que todavía no se tienen en cuenta

In [42]:
data["Garaje"] = data["Info_lower"].apply(lambda x : 1 if element_to_search in x else 0)

In [43]:
garaje_comment = data.apply(lambda x : search_context(element_to_search, x["Comentario_lower"], 1) if element_to_search not in x["Info_lower"] and element_to_search in x["Comentario_lower"] else "0", axis = 1)
num_ocurrence = np.unique(garaje_comment, return_counts=True)
for item, sentence in enumerate(num_ocurrence[0]):
    print(f'{sentence} appears {num_ocurrence[1][item]}')

 garaje appears 11
(dia/noche), garaje appears 1
- garaje appears 1
/ garaje appears 4
0 appears 18289
2 garaje appears 1
3). garaje appears 1
a garaje appears 14
adquirir garaje appears 3
al garaje appears 12
alquilar garaje appears 5
amplio garaje appears 2
ascensor, garaje appears 4
ascensor. garaje appears 1
baja: garaje appears 1
bano. garaje appears 1
banos, garaje appears 3
blindada, garaje appears 1
calderas, garaje appears 1
casa, garaje appears 1
central garaje appears 1
cocina. garaje appears 1
como garaje appears 2
comodo garaje appears 1
comunicado. garaje appears 1
con garaje appears 22
con, garaje appears 1
con: garaje appears 4
conductos, garaje appears 1
conserje garaje appears 1
de garaje appears 797
desde garaje appears 1
despensa. garaje appears 1
diferentes garaje appears 1
edificio garaje appears 3
edificio. garaje appears 2
el garaje appears 7
electrodomesticos. garaje appears 1
empotrados. garaje appears 1
en garaje appears 1
energetico. garaje appears 1
finca. 

Aunque continuemos con el mismo dilema que previamente en los comentarios, existe una conjugación que posee la mayoría de los casos, "de garaje". Se realiza el estudio para ese en concreto

In [44]:
garaje_comment = data.apply(lambda x : search_context("de garaje", x["Comentario_lower"], 2) if element_to_search not in x["Info_lower"] and element_to_search in x["Comentario_lower"] else "0", axis = 1)
num_ocurrence = np.unique(garaje_comment, return_counts=True)
for item, sentence in enumerate(num_ocurrence[0]):
    print(f'{sentence} appears {num_ocurrence[1][item]}')

 plaza de garaje appears 31
 plazas de garaje appears 4
 posibilidad de garaje appears 11
(2 plazas de garaje appears 1
(aeropuerto-bcn). posibilidad de garaje appears 1
(existen plazas de garaje appears 1
* plaza de garaje appears 1
- posibilidad de garaje appears 1
. plaza de garaje appears 1
0 appears 18289
1 "dispone de garaje appears 1
1 plaza de garaje appears 3
1 plazas de garaje appears 1
10m2 aprox. de garaje appears 1
2 plantas de garaje appears 1
2 plaza de garaje appears 1
2 plazas de garaje appears 28
2018- plaza de garaje appears 1
24h, posibilidad de garaje appears 1
24h. plaza de garaje appears 1
3 plazas de garaje appears 12
4 plazas de garaje appears 8
5.000 plazas de garaje appears 4
5.29%. posibilidad de garaje appears 1
50. plaza de garaje appears 1
72 m2 de garaje appears 1
90/mes plaza de garaje appears 1
FAIL appears 235
a plaza de garaje appears 5
a/a. plaza de garaje appears 1
adquirir plaza de garaje appears 21
adquirir plaza/s de garaje appears 1
adquirir pl

Se observa que en muchos casos no se incluye la plaza de garaje con a vivienda, sino que se comunica que se puede solicitar a mayores. Este trabajo de diferenciar cual es el caso de cada uno es demasiado tedioso, por lo que se descarta seguir estudiándolo

## Habitaciones
Comprobación de en donde podemos encontrar el elemento habitaciones

In [45]:
element_to_search = 'hab'
## Comprobación de lo ya rellenado
print(sum(data["N Habitaciones"].apply(lambda x : 1 if x != 'Desconocido' else 0)))
## Comprobación de lo que falta por rellenar
print(sum(data["N Habitaciones"].apply(lambda x : 1 if x == 'Desconocido' else 0)))
## Comprobación campo Info
print(sum(data["Info_lower"].apply(lambda x : 1 if element_to_search in x else 0)))
## Comprobación campo Mas detalles, sin tenr en cuenta los encontrados en Info
print(sum(data.apply(lambda x : 1 if element_to_search in x["Mas detalles_lower"] and element_to_search not in x["Info_lower"] else 0, axis = 1)))
## Comprobación campo Comentario, sin tenr en cuenta los encontrados en Mas detalles
print(sum(data.apply(lambda x : 1 if element_to_search in x["Comentario_lower"] and element_to_search not in x["Mas detalles_lower"] else 0, axis = 1)))

18703
648
18703
648
0


Observando estos resultados intuimos que los pocos casos que quedan sin rellenar del número de habitaciones, seguramente sea posible sacarlos del campo "Mas detalles"

In [46]:
hab_comment = data.apply(lambda x : search_context(element_to_search, x["Mas detalles_lower"], 3) if element_to_search not in x["Info_lower"]  else "0", axis = 1)
num_ocurrence = np.unique(hab_comment, return_counts=True)
for item, sentence in enumerate(num_ocurrence[0]):
    print(f'{sentence} appears {num_ocurrence[1][item]}')

0 appears 18703
m2 construidos, sin hab appears 356
m2 utiles, sin hab appears 292


Por lo visto, esas entradas no poseen habitaciones construidas, por lo que le podemos asignar valor 0

In [47]:
data.loc[data["N Habitaciones"] == "Desconocido", "N Habitaciones"] = 0

## m2
Comprobación del elemento m2

In [48]:
element_to_search = 'm2'
## Comprobación de lo ya rellenado
print(sum(data["m2"].apply(lambda x : 1 if x != 'Desconocido' else 0)))
## Comprobación de lo que falta por rellenar
print(sum(data["m2"].apply(lambda x : 1 if x == 'Desconocido' else 0)))
## Comprobación campo Info
print(sum(data["Info_lower"].apply(lambda x : 1 if element_to_search in x else 0)))
## Comprobación campo Mas detalles, sin tenr en cuenta los encontrados en Info
print(sum(data.apply(lambda x : 1 if element_to_search in x["Mas detalles_lower"] and element_to_search not in x["Info_lower"] else 0, axis = 1)))
## Comprobación campo Comentario, sin tenr en cuenta los encontrados en Mas detalles
print(sum(data.apply(lambda x : 1 if element_to_search in x["Comentario_lower"] and element_to_search not in x["Mas detalles_lower"] else 0, axis = 1)))

19351
0
19351
0
0


m2 está totalmente rellenado

## Baños
Comprobación de los baños

In [49]:
elment_to_search = 'bano'
## Comprobación campo Info
print(sum(data["Info_lower"].apply(lambda x : 1 if elment_to_search in x else 0)))
## Comprobación campo Mas detalles, sin tenr en cuenta los encontrados en Info
print(sum(data.apply(lambda x : 1 if elment_to_search in x["Mas detalles_lower"] and elment_to_search not in x["Info_lower"] else 0, axis = 1)))
## Comprobación campo Comentario, sin tenr en cuenta los encontrados en Mas detalles
print(sum(data.apply(lambda x : 1 if elment_to_search in x["Comentario_lower"] and elment_to_search not in x["Mas detalles_lower"] else 0, axis = 1)))

0
19351
0


In [50]:
bano_comment = data.apply(lambda x : search_context(elment_to_search, x["Mas detalles_lower"], 1) if elment_to_search not in x["Info_lower"]  else "0", axis = 1)
num_ocurrence = np.unique(bano_comment, return_counts=True)
for item, sentence in enumerate(num_ocurrence[0]):
    print(f'{sentence} appears {num_ocurrence[1][item]}')

1 bano appears 9375
10 bano appears 18
11 bano appears 6
12 bano appears 2
13 bano appears 2
14 bano appears 2
16 bano appears 3
17 bano appears 1
2 bano appears 5964
3 bano appears 2017
4 bano appears 911
41 bano appears 1
5 bano appears 570
6 bano appears 249
7 bano appears 125
8 bano appears 59
9 bano appears 34
sin bano appears 12


Este resultadonos confirma que se pueden sacar todos los resultados de forma simple de Mas detalles para conocer el número de baños

In [51]:
data['N banos'] = data['Mas detalles_lower'].apply(lambda x: re.search('(\d{1,}) bano', x)[1] if re.search('(\d{1,}) bano', x) is not None else 0)
data.loc[data['N banos'] == 'sin', 'N banos'] = 0
data['N banos'] = data['N banos'].astype(int)

In [52]:
num_ocurrence = np.unique(data['N banos'], return_counts=True)
for item, sentence in enumerate(num_ocurrence[0]):
    print(f'{sentence} appears {num_ocurrence[1][item]}')

0 appears 12
1 appears 9375
2 appears 5964
3 appears 2017
4 appears 911
5 appears 570
6 appears 249
7 appears 125
8 appears 59
9 appears 34
10 appears 18
11 appears 6
12 appears 2
13 appears 2
14 appears 2
16 appears 3
17 appears 1
41 appears 1


Todos los matches quedan bien rellenados

## Campos Latitud y Longitud

Para obtener los campos de latitud y longitud, buscaremos las coordenadas del punto medio de cada subzona y las guardaremos en dos diccionarios, uno de latitud y otro de longitud. Tras esto se crearán y cubrirán los campos respectivos mediante el uso de estos dos elementos.

In [53]:
# Diccionario de latitud por zonas

dict_latitud = {
'Malasaña-Universidad':40.4263 , 'Lavapiés-Embajadores':40.4089, 'Palacio':40.4152, 'Chueca-Justicia':40.4243,
'Sol':40.4173,'Huertas-Cortes':40.4139,'Alameda de Osuna':40.4567,'Casco Histórico de Barajas':40.4744,
'Timón':40.4733,'Campo de las Naciones-Corralejos':40.4560,'Aeropuerto':40.4604,'Peñagrande':40.4787,
'Las Tablas':40.5039,'Tres Olivos - Valverde':40.4969,'Pilar':40.4770,'Mirasierra':40.4936,
'La Paz':40.4822,'Fuentelarreina':40.4803,'Montecarmelo':40.5085,'Arroyo del Fresno':40.4901,
'El Pardo':40.5205,'El Viso':40.4448,'Bernabéu-Hispanoamérica':40.4579,'Nueva España':40.4632,
'Prosperidad':40.4454,'Castilla':40.4757,'Ciudad Jardín':40.4487,'Pueblo Nuevo':40.4275,
'Ventas':40.4255,'Quintana':40.4365,'Concepción':40.4389,'Costillares':40.4761,
'San Juan Bautista':40.4511,'San Pascual':40.4433,'Colina':40.4584,'Atalaya':40.46474,
'Cuatro Caminos':40.4519,'Berruguete':40.4598,'Valdeacederas':40.4674,'Cuzco-Castillejos':40.4628,
'Bellas Vistas':40.45298,'Ventilla-Almenara':40.4711,'Argüelles':40.4281,'Aravaca':40.4564,
'Ciudad Universitaria':40.4548,'Valdemarín':40.4682,'El Plantío':40.4696,'Valdezarza':40.4660,
'Casa de Campo':40.4193,'Recoletos':40.4253,'Goya':40.4252,'Castellana':40.4334,
'Lista':40.43215,'Guindalera':40.4364,'Fuente del Berro':40.4250,'Almagro':40.4335,
'Trafalgar':40.4335,'Nuevos Ministerios-Ríos Rosas':40.4449,'Gaztambide':40.4354,'Vallehermoso':40.4427,
'Arapiles':40.4345,'Pacífico':40.4047,'Jerónimos':40.4154,'Ibiza':40.4191,
'Niño Jesús':40.4118,'Adelfas':40.4007,'Estrella':40.4134,'Puerta del Ángel':40.4102,
'Aluche':40.3921,'Lucero':40.4024,'Águilas':40.3813,'Campamento':40.3842,
'Los Cármenes':40.4019,'Cuatro Vientos':40.3720,'Vista Alegre':40.3840,'San Isidro':40.3960,
'Puerta Bonita':40.38161,'Opañel':40.3912,'Buena Vista':40.3681,'Abrantes':40.3810,
'Comillas':40.3934,'Pau de Carabanchel':40.3753,'Palos de Moguer':40.4038,'Imperial':40.4073,
'Delicias':40.3961,'Acacias':40.4015,'Chopera':40.4017,'Legazpi':40.3919,
'Almendrales':40.3837,'Moscardó':40.3892,'San Fermín':40.3713,'12 de Octubre-Orcasur':40.3696,
'Orcasitas':40.3687,'Pradolongo':40.3815,'Zofío':40.3802,'San Diego':40.3917,
'Numancia':40.4022,'Palomeras sureste':40.3870,'Entrevías':40.3781,'Palomeras Bajas':40.3867,
'Portazgo':40.3904,'Ensanche de Vallecas - La Gavia':40.3725,'Casco Histórico de Vallecas':40.3763,'Santa Eugenia':40.3828,
'Media Legua':40.4118,'Marroquina':40.4109,'Vinateros':40.4058,'Fontarrón':40.4009,'Pavones':40.4003,
'Horcajo':40.4074,'El Cañaveral - Los Berrocales':40.4059,'Casco Histórico de Vicálvaro':40.4017,'Ambroz':40.4082,
'Valdebernardo - Valderribas':40.3964,'Simancas':40.4361,'Rejas':40.4448,'Canillejas':40.4446,
'Arcos':40.4213,'Rosas':40.4292,'Salvador':40.4436,'Amposta':40.4270,
'Hellín':40.4315,'Villaverde Alto':40.3439,'Los Rosales':40.3576,'San Cristóbal':40.3419,
'Los Ángeles':40.3567,'Butarque':40.3445,'Sanchinarro':40.4908,'Conde Orgaz-Piovera':40.4551,
'Pinar del Rey':40.4738,'Canillas':40.4646,'Valdebebas - Valdefuentes':40.4875,'Virgen del Cortijo - Manoteras':40.4879,
'Apóstol Santiago':40.4773,'Palomas':40.4531}

In [54]:
# Diccionario de longitud por zonas

dict_longitud = {
'Malasaña-Universidad':-3.7066, 'Lavapiés-Embajadores':-3.7002, 'Palacio':-3.7110, 'Chueca-Justicia':-3.6989,
'Sol':-3.7045,'Huertas-Cortes':-3.6985,'Alameda de Osuna':-3.5914,'Casco Histórico de Barajas':-3.5784,
'Timón':-3.5909,'Campo de las Naciones-Corralejos':-3.6188,'Aeropuerto':-3.5853,'Peñagrande':-3.7272,
'Las Tablas':-3.6718,'Tres Olivos - Valverde':-3.6922,'Pilar':-3.7098,'Mirasierra':-3.7157,
'La Paz':-3.6969,'Fuentelarreina':-3.7411,'Montecarmelo':-3.6968,'Arroyo del Fresno':-3.7275,
'El Pardo':-3.7771,'El Viso':-3.6854,'Bernabéu-Hispanoamérica':-3.6819,'Nueva España':-3.6780,
'Prosperidad':-3.6669,'Castilla':-3.6707,'Ciudad Jardín':-3.6732,'Pueblo Nuevo':-3.6365,
'Ventas':-3.6497,'Quintana':-3.6457,'Concepción':-3.6497,'Costillares':-3.6687,
'San Juan Bautista':-3.6564,'San Pascual':-3.6525,'Colina':-3.6600,'Atalaya':-3.6650,
'Cuatro Caminos':-3.6968,'Berruguete':-3.7049,'Valdeacederas':-3.7033,'Cuzco-Castillejos':-3.6958,
'Bellas Vistas':-3.7069,'Ventilla-Almenara':-3.6945,'Argüelles':-3.7172,'Aravaca':-3.7775,
'Ciudad Universitaria':-3.7347,'Valdemarín':-3.7787,'El Plantío':-3.8162,'Valdezarza':-3.7161,
'Casa de Campo':-3.7911,'Recoletos':-3.6850,'Goya':-3.6737,'Castellana':-3.6830,
'Lista':-3.6750,'Guindalera':-3.6665,'Fuente del Berro':-3.6633,'Almagro':-3.6920,
'Trafalgar':-3.7008,'Nuevos Ministerios-Ríos Rosas':-3.6927,'Gaztambide':-3.7144,'Vallehermoso':-3.7113,
'Arapiles':-3.7080,'Pacífico':-3.6775,'Jerónimos':-3.6898,'Ibiza':-3.6742,
'Niño Jesús':-3.6732,'Adelfas':-3.6714,'Estrella':-3.6648,'Puerta del Ángel':-3.7305,
'Aluche':-3.7543,'Lucero':-3.7500,'Águilas':-3.7710,'Campamento':-3.7989,
'Los Cármenes':-3.7359,'Cuatro Vientos':-3.7679,'Vista Alegre':-3.7453,'San Isidro':-3.7279,
'Puerta Bonita':-3.7417,'Opañel':-3.7218,'Buena Vista':-3.7457,'Abrantes':-3.7289,
'Comillas':-3.7108,'Pau de Carabanchel':-3.7443,'Palos de Moguer':-3.6940,'Imperial':-3.7172,
'Delicias':-3.6912,'Acacias':-3.7069,'Chopera':-3.7102,'Legazpi':-3.7214,
'Almendrales':-3.6996,'Moscardó':-3.7069,'San Fermín':-3.6898,'12 de Octubre-Orcasur':-3.7009,
'Orcasitas':-3.7104,'Pradolongo':-3.7078,'Zofío':-3.7145,'San Diego':-3.6675,
'Numancia':-3.6607,'Palomeras sureste':-3.6408,'Entrevías':-3.6681,'Palomeras Bajas':-3.6574,
'Portazgo':-3.6481,'Ensanche de Vallecas - La Gavia':-3.6115,'Casco Histórico de Vallecas':-3.6260,'Santa Eugenia':-3.6137,
'Media Legua':-3.6568,'Marroquina':-3.6458,'Vinateros':-3.6425,'Fontarrón':-3.6442,'Pavones':-3.6314,
'Horcajo':-3.6284,'El Cañaveral - Los Berrocales':-3.5525,'Casco Histórico de Vicálvaro':-3.6193,'Ambroz':-3.6093,
'Valdebernardo - Valderribas':-3.6081,'Simancas':-3.6235,'Rejas':-3.5779,'Canillejas':-3.6098,
'Arcos':-3.6169,'Rosas':-3.6071,'Salvador':-3.6334,'Amposta':-3.6213,
'Hellín':-3.6147,'Villaverde Alto':-3.7084,'Los Rosales':-3.6872,'San Cristóbal':-3.6880,
'Los Ángeles':-3.6983,'Butarque':-3.6756,'Sanchinarro':-3.6574,'Conde Orgaz-Piovera':-3.6379,
'Pinar del Rey':-3.6453,'Canillas':-3.6411,'Valdebebas - Valdefuentes':-3.6104,'Virgen del Cortijo - Manoteras':-3.6654,
'Apóstol Santiago':-3.6592,'Palomas':-3.6148}

In [55]:
# Ahora recorremos el diccionario de latitud y asignamos los valores 
# numéricos correspondientes a cada subzona

for key,item in dict_latitud.items():
    data.loc[data['Subzona'] == key, 'Latitud'] = item

In [56]:
# Ahora recorremos el diccionario de longitud y asignamos los valores 
# numéricos correspondientes a cada subzona

for key,item in dict_longitud.items():
    data.loc[data['Subzona'] == key, 'Longitud'] = item

In [57]:
data.sample(3)

,ID,Precio,Tipo,N Habitaciones,m2,Comentario,Zona,Subzona,Info,Mas detalles,...,Mas detalles_lower,Info_lower,Planta,Cantidad de plantas,Piscina,Ascensor,Garaje,N banos,Latitud,Longitud
18573,96900683,160.000,Piso,3,78,"[Piso de 78 m2 en planta (8), con vistas, tres...",Villaverde,Los Ángeles,78 m² 3 hab. Planta 8ª exterior con asce...,"[78 m² construidos, 3 habitaciones, 1 baño, Te...",...,"b'[78 m2 construidos, 3 habitaciones, 1 bano, ...",b' 78 m2 3 hab. planta 8a exterior con as...,8,1,0,1,0,1,40.3567,-3.6983
8293,96831259,1.900.000,Piso,5,210,"[Piso TOP con terraza soleada, orientación Sur...",Barrio de Salamanca,Recoletos,210 m² 5 hab. Planta 5ª exterior con asc...,"[210 m² construidos, 5 habitaciones, 3 baños, ...",...,"b'[210 m2 construidos, 5 habitaciones, 3 banos...",b' 210 m2 5 hab. planta 5a exterior con a...,5,1,0,1,1,3,40.4253,-3.6850
12141,97376201,205.000,Piso,3,85,[PARTICULAR. Vende piso magníficamente situad...,Latina,Aluche,85 m² 3 hab. Planta 8ª exterior con asce...,"[85 m² construidos, 3 habitaciones, 1 baño, Te...",...,"b'[85 m2 construidos, 3 habitaciones, 1 bano, ...",b' 85 m2 3 hab. planta 8a exterior con as...,8,1,0,1,0,1,40.3921,-3.7543


## Campo Precio_m2 

En este campo crearemos un campo que sea precio/m2 resultante de la división de los campos con esos nombres. De esta manera obtendremos un campo que nos aproximen un precio de las viviendas de una manera más estandarizada y extrapolable a nuevos registros.

In [58]:
# Cambiamos el tipo de dato de m2 y Precio de string a numérico, vimos que no presentaba valores que no fuesen de este tipo
data['m2'] = data['m2'].map(lambda x: x.replace('.','')) # Reemplazamos los puntos para convertir a numérico
data['m2'] = data['m2'].astype(int)
data['Precio'] = data['Precio'].map(lambda x: x.replace('.','')) # Reemplazamos los puntos para convertir a numérico
data['Precio'] = data['Precio'].astype(int)
data['N Habitaciones'] = data['N Habitaciones'].astype(int)
data['Planta'] = data['Planta'].astype(int)

data['Precio_m2'] = data['Precio'].div(data['m2'].values,axis=0)

In [59]:
data.dtypes

ID                       int64
Precio                   int64
Tipo                    object
N Habitaciones           int64
m2                       int64
Comentario              object
Zona                    object
Subzona                 object
Info                    object
Mas detalles            object
Comentario_lower        object
Mas detalles_lower      object
Info_lower              object
Planta                   int64
Cantidad de plantas      int64
Piscina                  int64
Ascensor                 int64
Garaje                   int64
N banos                  int64
Latitud                float64
Longitud               float64
Precio_m2              float64
dtype: object

In [60]:
data.to_csv('Datos_Madrid_tratados.csv',index=False)